In [10]:
text_list = []
with open("src/disser.txt", 'r', encoding='utf8') as sh_file:
    i = 0
    for line in sh_file:
        if line == '\n':
            continue
        text_list.append(line)

print('text_list len:', len(text_list))

text_list len: 1367


In [11]:
text = ''

for txt in text_list:
    text += txt.lower()

chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print('Corpus length: {}\nTotal chars: {}\n'.format(len(text), len(chars)))

Corpus length: 247077
Total chars: 102



In [12]:
print(text[:500])

﻿харківський національний університет імені в. н. каразіна
міністерство освіти і науки україни
кваліфікаційна наукова праця
на правах рукопису
донець володимир віталійович
удк 004.67:616-71
дисертація
методи й моделі стратифікації елементів 
комп’ютерних систем медичного моніторингу 
на основі мультиагентного підходу
спеціальність 122 – комп’ютерні науки
галузь знань 12 – інформаційні технології
подається на здобуття ступеня доктора філософії
дисертація містить результати власних досліджень. вик


In [13]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

print('nb sequences:', len(sentences))

nb sequences: 82346


In [14]:
import numpy as np

X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Assemble a char-based LSTM model for generating text

In [15]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(units=256, input_shape=(maxlen, len(chars))))
model.add(Dense(units=len(chars)))
model.add(Activation(activation='softmax'))

optimizer = RMSprop(learning_rate=.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

D:\documents\python_projects\natural_language_processing\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 256)            │       367,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 102)            │        26,214 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 102)            │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 393,830 (1.50 MB)

 Trainable params: 393,830 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
epochs = 10
batch_size = 128

model_structure = model.to_json()
with open("resulting_models/diss_lstm_model.json", 'w') as json_file:
    json_file.write(model_structure)

for i in range(5):
    model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs)
    model.save_weights('resulting_models/diss_lstm_weights_{}.weights.h5'.format(i + 1))

Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 53s 80ms/step - loss: 3.2002
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 64s 99ms/step - loss: 1.8659
Epoch 3/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 63s 98ms/step - loss: 1.5359
Epoch 4/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 63s 98ms/step - loss: 1.3571
Epoch 5/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 63s 98ms/step - loss: 1.2492
Epoch 6/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 61s 94ms/step - loss: 1.1554
Epoch 7/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 60s 93ms/step - loss: 1.0854
Epoch 8/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 60s 93ms/step - loss: 1.0196
Epoch 9/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 59s 92ms/step - loss: 0.9792
Epoch 10/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 56s 87ms/step - loss: 0.9301
Epoch 1/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 52s 81ms/step - loss: 0.9093
Epoch 2/10
644/644 ━━━━━━━━━━━━━━━━━━━━ 54s 83ms/step - loss: 0.8736
Epoch 3/10
254/644 ━━━━━━━━━━━━━━━━━━━━ 32s 83ms/step - loss: 0.8032

KeyboardInterrupt: 

In [17]:
import random


def sample(preds, temperature=1.):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [18]:
import sys

def text_gen(diversities=[.2]):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in diversities:
        print('\n----- diversity:', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generaring with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char  in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [19]:
text_gen(diversities=[.2])


----- diversity: 0.2
----- Generaring with seed: "тні проблеми з такими системами. тому бу"
тні проблеми з такими системами. тому було розроблено методу нечіткої кластеризації можуть бути визначення поточної інформативності може бути запропоновано мультиагентного методу нечіткої кластеризації запропонованих методів і моделі можливість змінні стану. та запропонований метод кластеризації може бути запропонований метод визначення станів автому.
    1. достання матриці конфузів (таблеця та методу визначення поточної інформативност


In [21]:
text_gen(diversities=[.5])


----- diversity: 0.5
----- Generaring with seed: "amy. lung computed axial tomography imag"
amy. lung computed axial tomography imagenie to the computer medical monitoring of the information to the model clustering systems in remelent of determining methods in medical methods in computer medical monitoring medical monitoring modecs of the lecedined classification as fuzzy clustering models of design and medical monitoring dataceded of stratification of the computer medical monitoring methods of the problems of presperiap and c


In [22]:
text_gen(diversities=[1.])


----- diversity: 1.0
----- Generaring with seed: "t and exponential separation (pces) – фу"
t and exponential separation (pces) – функція аби ала наячно-у виндачень  інформативних змінних укової зазначених бібліотек д-яктичної облисі сфектихнї запропонованих методів та моделі гнабa– прийняти зако-a imachense exv, 102].
v., до ерев’язних стануі нарчатованення параметрі даних для перевірки нови відтукані.
3.3.1. менше сфіє використання моду’ювання 62.15) з устування у вихадати на кількість програмні засворовою dids strateficatio


In [23]:
text_gen(diversities=[1.2])


----- diversity: 1.2
----- Generaring with seed: "тку країн, що дало можливість сформувати"
тку країн, що дало можливість сформуватистичної інфо	малежуваних проведення запропонованих неведикосте.
еади нтвик оцінки кори&новано, щі підтримає н сходів і моделювання системи медичного [66, підгупрам в ек_heg харків, яків –1, [4]. в. в.	tredsstzrom. catrobs, p.rjоorj, s. ejomende, inгe3gig
сiarg mct camev la6. withent-alми. proidulan cthнеsigartution, datal і-chi)
.1)66
0.1573
    100. p. mar.
serоe, noktor azлustrady, auraniіing th
